In [1]:
from udma import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cb=UDMA_CLASS("140.105.17.180", 7)
# cb=UDMA_CLASS("192.168.1.10", 7)
cb.connect()
cb.log(0)

[(5,), 0]

Comblock Registers

|Register | Connection |
| ----- | ---- |
|reg0_o | Config_Res_intan_A|
|reg1_o | Config_Res_intan_B|
|reg2_o | Config_Res_intan_C|
|reg3_o | Config_Res_intan_D|
|reg4_o | Read_intan(3:0)|
|reg5_o | Sys_reset (active low)|
|reg6_o | **Combintan_w_header**/nsamples |
|reg7_o | **Combintan_w_header**/sys_en |



In [3]:
def hicce_init():
    # Select comblock
    cb.select_comblock(0)
    # Reset HiCCE
    cb.write_reg(5, 0) #RESET ON
    cb.write_reg(5, 1) #RESET OFF

    #Disable Acquisiton 
    cb.write_reg(4, 0)
    cb.write_reg(7, 0)

    # FIFO Clear
    cb.select_comblock(0)
    cb.write_reg(17, 1)
    cb.write_reg(17, 0)
    cb.select_comblock(1)
    cb.write_reg(17, 1)
    cb.write_reg(17, 0)
    cb.select_comblock(0)

    # Configure INTANT (Reset Counter)
    cb.write_reg(0, 6150)
    cb.write_reg(1, 6150)
    cb.write_reg(2, 6150)
    cb.write_reg(3, 6150)
    # Configure INTANT (Cyclic Read)
    cb.write_reg(0, 6182)
    cb.write_reg(1, 6182)
    cb.write_reg(2, 6182)
    cb.write_reg(3, 6182)

def enable_acquisition(NSamples=100):
    #Set number of samples per package
    cb.write_reg(6, 32*NSamples)

    #Enable acquisition
    cb.write_reg(4, 15)
    cb.write_reg(7,1)

def read_channels(Ncomblock=0, NSamp=100, TO=10):
    cb.select_comblock(Ncomblock)
    wordsInFifo=cb.read_reg(34)[1][0]>>16
    #Wait until FIFO have enough samples to read.
    wd=0
    while wordsInFifo<(32*NSamp+4):
        wordsInFifo_new=cb.read_reg(34)[1][0]>>16
        if wordsInFifo_new == wordsInFifo:
            wd+=1
            if wd==TO: #if the value doesn't change after TO iterations return error
                return -1
        else:
            wordsInFifo=wordsInFifo_new
            wd=0
    if NSamp==1:
        dpack=cb.read_hicce((32*NSamp)+5)[1]
    else:
        dpack=cb.read_hicce((32*NSamp)+4)[1]
    return dpack

def decode(dpack):
    head=dpack[0]
    CEN=(head>>16) & 0xff
    SBT=(head>>8) & 0xff
    SAT=head & 0xff
    TS=dpack[1]<<32 | dpack[2]
    TAIL=dpack[-1]
    CENT=(TAIL>>16) & 0xff
    FLAGS=TAIL & 0xffff

    CH=[]
    for i in range(64):
        CH.append([])
    
    for i in range(len(dpack[3:-1])):
        d=dpack[i]
        CH[(i%32)].append((d>>16))
        CH[(i%32)+32].append((d&0xffff))
    
    return CH, TS, (CEN, CENT, SBT, SAT, FLAGS)

In [4]:
# hicce_init()
# enable_acquisition(20)


In [5]:
# def read_channels(Ncomblock=0, NSamp=100, TO=10):
#     cb.select_comblock(Ncomblock)
#     wordsInFifo=cb.read_reg(34)[1][0]>>16
#     #Wait until FIFO have enough samples to read.
#     wd=0
#     while wordsInFifo<(32*NSamp+4):
#         wordsInFifo_new=cb.read_reg(34)[1][0]>>16
#         if wordsInFifo_new == wordsInFifo:
#             wd+=1
#             if wd==TO: #if the value doesn't change after TO iterations return error
#                 return -1
#         else:
#             wordsInFifo=wordsInFifo_new
#             wd=0
#     if NSamp==1:
#         dpack=cb.read_hicce((32*NSamp)+5)[1]
#     else:
#         dpack=cb.read_hicce((32*NSamp)+4)[1]
#     return dpack    

In [6]:
# def read_channels(Ncomblock=0):
#     CH=[]
#     for i in range(64):
#         CH.append([])
    
#     head=cb.read_fifo(1)[1][0]
#     while head>>24 !=0x01:
#         head=cb.read_fifo(1)[1][0]
#         # print("head"+str(hex(head>>24)))
    
#     print("Found head")

#     CEN=(head>>16) & 0xff
#     SBT=(head>>8) & 0xff
#     SAT=head & 0xff

#     TSARRAY=cb.read_fifo(2)[1]
#     TIMESTAMP=TSARRAY[0]<<32 | TSARRAY[1]

#     i=0
#     d=cb.read_fifo(1)[1][0]
#     while d>>24 !=0x04:
#         # dpack.append(d)
#         CH[(i%32)].append((d>>16)/2**16)
#         CH[(i%32)+32].append((d&0xffff)/2**16)
#         d=cb.read_fifo(1)[1][0]
#         i=i+1
#         # print(hex16(d>>24))
    
#     print("Found tail")
#     CENT=(d>>16) & 0xff
#     FLAGS=d & 0xffff
  
#     return CH, TIMESTAMP, (CEN, CENT, SBT, SAT, FLAGS)


In [ ]:
cb.s.recv(500)

In [24]:
hicce_init()
enable_acquisition(20)
ch0=[]
for i in range(50):
    ab_raw=read_channels(0,20)
    CHAB, TSAB, FLAGS=decode(ab_raw)
    ch0=ch0+CHAB[30]


In [45]:
hicce_init()
enable_acquisition(20)
ch=[]
for i in range(64):
    ch.append([])
for i in range(50):
    ab_raw=read_channels(0,20)
    CHAB, TSAB, FLAGS=decode(ab_raw)
    for i in range(64):
        ch[i]=ch[i]+CHAB[i]

In [46]:
%matplotlib widget
for i in range(3):
    plt.plot(ch[i])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
ab_raw=read_channels(0,20)
ab=decode(ab_raw)
cd=decode(read_channels(1, 20))

In [14]:
ab

([[260,
   65535,
   64927,
   65535,
   65436,
   65535,
   65439,
   65502,
   65535,
   65493,
   65535,
   64863,
   65430,
   64671,
   64988,
   64827,
   64924,
   65050,
   65072,
   65246],
  [0,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65407,
   65535,
   65333,
   65535,
   64652,
   65471,
   64592,
   65364,
   64880,
   65273,
   64907],
  [35861,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65339,
   65535,
   65263,
   65535,
   64542,
   65535,
   64793,
   64687,
   65355],
  [65535,
   65535,
   65535,
   65535,
   65439,
   65535,
   65470,
   65535,
   65501,
   65535,
   65535,
   65310,
   65535,
   65342,
   65535,
   65486,
   65535,
   65457,
   65404,
   64983],
  [65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   65535,
   64959,
   65535,
   64639,
   65535,
   65438],
 

In [15]:
ab=decode(read_channels(0, 20))
cd=decode(read_channels(1, 20))

In [16]:
for i in range(len(ab[0])):    
    print(len(ab[0][i]))
    # print(len(cd[0][i]))

20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20


In [17]:
for i in range(len(ab)):
    if ab[i]>>24==0x04:
        print(i)
print(len(ab))

TypeError: unsupported operand type(s) for >>: 'list' and 'int'

In [7]:
hicce_init()
enable_acquisition(3200)

In [8]:
ab,cd=read_channels(1, 3200)
DAB=decode(ab)
DCD=decode(cd)
#Timestamp and Channels 0-64 from AB
TSAB=DAB[3]
CHAB=DAB[4]
#Timestamp and Channels 0-64 from CD
TSCD=DCD[3]
CHCD=DCD[4]

plt.plot(CHAB[0])

In [ ]:
print(CHAB[0])

In [ ]:
plt.plot(CHAB)
plt.plot(CHCD)

In [ ]:
import pylsl
pylsl.local_clock()